In [1]:
!pip install vaderSentiment
!pip install shap --no-deps

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/126.0 kB ? eta -:--:--
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 4.7 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
import re
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, VotingClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from transformers import BertTokenizer, BertForSequenceClassification
import torch
import torch.nn.functional as F
from tqdm import tqdm

In [3]:
pd.set_option('display.max_columns', None)

In [4]:
df1 = pd.read_csv('/content/Combined_News_S&P500.csv')
df1.head()

,Date,Label,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,Top9,Top10,Top11,Top12,Top13,Top14,Top15,Top16,Top17,Top18,Top19,Top20,Top21,Top22,Top23,Top24,Top25
0,08/02/2013,1,An Egyptian Salafi preacher said raping and se...,Australian children are to be banned from blow...,35 ancient pyramids discovered in Sudan,A Somali court has sentenced a woman to a year...,"Woman vandalizes famous Louvre painting ""Liber...","The drone war is a shadow war, widely reported...",Aviation experts say new Iranian stealth jet i...,Police defuse bomb in Madrid cathedral,All dogs will now have to be microchipped in E...,Spain Offers Citizenship to Descendants of Jew...,The brutal gang rape and murder of a young wom...,Personal finance education to be compulsory ac...,UN says bribe payments soar in Afghanistan - S...,Child-free seats on AirAsia for just 25 extra,Vermont Maple Sugar Candy Producer implicated ...,Born In An Iranian Prison -- And Into A Lifeti...,Back in the USSR? Key Soviet document that pro...,Blunt Words on China from U.S. Navy,Suicide bomber blows himself up near Mali sold...,Turkey tells US envoy to stop meddling in its ...,35 pyramids discovered in a Sudan Necropolis,"Derision, bafflement over Egypt prime minister...",A Sri Lankan prisoner who tried to hide his ce...,Gunmen Kill 9 in Attacks on Nigerian Polio Cli...,"Overwhelmed by a critical housing crisis, Hong..."
1,11/02/2013,0,Pope Pope Benedict XVI 'to resign'. The Pope i...,"To get the gold, they will have to kill every ...",\nLeaked Documents Reveal Gas Company Eyeing P...,China tightens concert rules after Elton John'...,"""Unfortunately, we live in an age where any no...",An asteroid is going to fly by Earth on Feb.15...,Wind power capacity grew 20% globally in 2012,Farmers use of genetically modified soybeans g...,Fascism mounts a comeback in Italy. As Italy p...,Outrage After Job Center Suggests Brothel Job ...,"Afghan boy, 13, jailed for having sex with men","Microsoft, Apple (and others) summonsed to exp...",An energy company is eyeing up the gas reserve...,Israeli police detained 10 women at one of Jud...,Attackers in northern Nigeria killed three Kor...,Islamic extremists armed with AK-47 automatic ...,"After 2 months absent, still no sign of Chavez...",Nigerian prostitutes promise week of free sex ...,"To curb poaching menace, India is considering ...","Anti-gay, religious group gets funds from the ...",Two people in southwest China have tested posi...,China has leapfrogged the US to become the wor...,"Over the Antarctic, the Smallest Ozone Hole In...","The pilgrims came, millions upon millions of t...",Pope Benedict XVI to resign feb 28
2,12/02/2013,1,Iran urges to destroy all atomic weapons after...,China's patience with North Korea wears thin a...,"China's New Bachelor Class ""An estimated 12 to...","4.9 earthquake detected in North Korea, indica...","In surprise move, IOC drops wrestling from 202...",North Korea tells U.N. will never bow to nucle...,The same international body that enacted a 199...,Healthy Vancouver senior commits suicide in bi...,Iran's 'fake' fighter jet which experts say ca...,Chinese Hackers Are Going After U.S. in Genera...,"North Korea: Nuke test just a ""first response""...","North Korea earthquake was a nuclear test, UN ...",Police hunting serial rapist dont know whom to...,Pope's Sudden Resignation Sends Shockwaves Thr...,Australia's ABC exposes the mystery of Prisone...,"Obama to announce return of 34,000 troops from...",Government killing online surveillance bill,A multibillion-dollar expansion of the Panama ...,Japan to send planes to monitor radiation afte...,North Korea withdraws from nuclear test site: ...,Olympic Committee Drops Wrestling From 2020 Ga...,Coca-Cola rejects findings that its product wa...,Not only has Putin made Russia the worlds larg...,"'Tories no longer party of privilege', claims ...",Non-muslim teachers at Islamic College forced ...
3,13/02/2013,0,The European Union and the US will begin forma...,'He will stay 

In [5]:
df2 = pd.read_csv("/content/all_stocks_3yr.csv")

def price_label(row):
    if row['Close'] >= row['Open']:
        return 1   # went up/stayed the same
    else:
        return 0   # went down

df2['price_label'] = df2.apply(price_label, axis=1)

df2

,Date,Open,High,Low,Close,Volume,Name,price_label
0,08/02/2013,15.07,15.12,14.630,14.75,8407500,AAL,0
1,11/02/2013,14.89,15.01,14.260,14.46,8882000,AAL,0
2,12/02/2013,14.45,14.51,14.100,14.27,8126000,AAL,0
3,13/02/2013,14.30,14.94,14.250,14.66,10259500,AAL,1
4,14/02/2013,14.94,14.96,13.160,13.99,31879900,AAL,0
...,...,...,...,...,...,...,...,...
406595,27/06/2016,46.30,46.59,45.280,45.42,5047270,ZTS,0
406596,28/06/2016,45.90,46.42,45.675,46.42,3628498,ZTS,1
406597,29/06/2016,46.88,47.38,46.680,47.24,3603850,ZTS,1
406598,30/06/2016,47.19,47.53,46.970,47.46,3769037,ZTS,1


In [6]:
df1.shape, df2.shape

((856, 27), (406600, 8))

In [7]:
headline = []
for row in range(0,len(df1.index)):
    headline.append(" ".join(str(x) for x in df1.iloc[row, 2:27]))

In [8]:
headline[0]

'An Egyptian Salafi preacher said raping and sexually harassing women protesters in Cairos Tahrir Square is justified, calling them crusaders who have no shame, no fear and not even feminism. Australian children are to be banned from blowing out candles on communal birthday cakes under new hygiene regulations 35 ancient pyramids discovered in Sudan A Somali court has sentenced a woman to a year in prison after she accused security forces of raping her. Woman vandalizes famous Louvre painting "Liberty Leading the People" by Delacroix The drone war is a shadow war, widely reported in the media but officially unacknowledged by the CIA and the White House. no one knows precisely how many casualties our shadow war has caused, but media and NGO reports suggest that the number of deaths is somewhere between 3,000 and 5,000. Aviation experts say new Iranian stealth jet is a pathetic hoax that cant even fly Police defuse bomb in Madrid cathedral All dogs will now have to be microchipped in Engl

In [9]:
import re
import spacy
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS

# Load spaCy model (make sure you've installed: python -m spacy download en_core_web_sm)
nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])

# Keep negation words like "not", "no", "never"
negation_words = {'not', 'no', 'nor', 'never', "n't"}
custom_stopwords = ENGLISH_STOP_WORDS.difference(negation_words)

# Clean + Lemmatize headlines (loop style)
clean_headline = []
for i in range(len(headline)):
    text = str(headline[i]).lower()                          # Lowercase
    text = re.sub(r"b[(')]", '', text)                       # Remove b'
    text = re.sub(r"b[(\")]", '', text)                      # Remove b"
    text = re.sub(r"\\'", '', text)                          # Remove \'
    text = re.sub(r"[\'\"]", '', text)                       # Remove ' and "
    text = re.sub(r'\bb\b', '', text)                        # Remove standalone 'b'
    text = re.sub(r'[^a-z\s]', '', text)                     # Remove punctuation/numbers

    # Tokenize + Lemmatize
    doc = nlp(text)
    words = []
    for token in doc:
        if token.lemma_ not in custom_stopwords and token.is_alpha:  # Stopwords & keep only alphabetic
            words.append(token.lemma_)

    text = ' '.join(words)
    text = re.sub(r'\s+', ' ', text).strip()                 # Remove extra spaces
    clean_headline.append(text)


In [10]:
clean_headline[0]

'egyptian salafi preacher say rape sexually harass woman protester cairo tahrir square justify crusader no shame no fear not feminism australian child ban blow candle communal birthday cake new hygiene regulation ancient pyramid discover sudan somali court sentence woman year prison accuse security force rape woman vandalize famous louvre paint liberty lead people delacroix drone war shadow war widely report medium officially unacknowledge cia white house no know precisely casualty shadow war cause medium ngo report suggest number death aviation expert say new iranian stealth jet pathetic hoax not fly police defuse bomb madrid cathedral dog microchippe england spain offer citizenship descendant jews force inquisition brutal gang rape murder young woman saturday lead outpouring anger south african frustrate national epidemic sexual violence personal finance education compulsory england say bribe payment soar afghanistan survey bn twice nation domestic revenue pay bribe childfree seat ai

In [11]:
df1["Combined_News"] = clean_headline
df1 = df1[["Date","Combined_News"]]
df1

,Date,Combined_News
0,08/02/2013,egyptian salafi preacher say rape sexually har...
1,11/02/2013,pope pope benedict xvi resign pope resign end ...
2,12/02/2013,iran urge destroy atomic weapon nkorea nuclear...
3,13/02/2013,european union begin formal talk freetrade agr...
4,14/02/2013,totally drugresistant tb uncovered outbreak un...
...,...,...
851,27/06/2016,barclay rbs share suspend trading tank pope sa...
852,28/06/2016,scientist australia want save great barrier re...
853,29/06/2016,explosion airport istanbul yemeni president te...
854,30/06/2016,jamaica propose marijuana dispenser tourist ai...


In [12]:
# ===============================
# Imports
# ===============================
import pandas as pd
import spacy
import nltk
from nltk.corpus import wordnet as wn
from sentence_transformers import SentenceTransformer, util
from spacy.lang.en.stop_words import STOP_WORDS
from nltk.util import ngrams
import torch

# ===============================
# NLTK downloads
# ===============================
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

# ===============================
# Load models
# ===============================
nlp = spacy.load("en_core_web_sm")
model = SentenceTransformer("all-MiniLM-L6-v2")
# device handling
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

# ===============================
# Custom stopwords (keep negatives)
# ===============================
negatives = {'no', 'not', 'never', "n't", "nor"}
custom_stopwords = STOP_WORDS - negatives

# ===============================
# N-gram functions
# ===============================
def generate_ngrams(tokens, n=2):
    return [' '.join(gram) for gram in ngrams(tokens, n)]

def check_ngram_ambiguity(sentence, n=2):
    """Return ambiguous n-grams from sentence."""
    doc = nlp(sentence)
    tokens = [token.text for token in doc
              if token.pos_ in ['NOUN', 'VERB', 'ADJ']
              and token.text.lower() not in custom_stopwords]

    ambiguous_ngrams = []
    for ng in generate_ngrams(tokens, n):
        senses = [wn.synsets(word) for word in ng.split()]
        if any(len(s) > 2 for s in senses):
            ambiguous_ngrams.append(ng)
    return ambiguous_ngrams

def ngram_semantic_score(sentence, n=2):
    """Calculate semantic ambiguity score for n-grams."""
    doc = nlp(sentence)
    tokens = [token.text for token in doc
              if token.pos_ in ['NOUN', 'VERB', 'ADJ']
              and token.text.lower() not in custom_stopwords]

    ngram_list = generate_ngrams(tokens, n)
    scores = []

    for ng in ngram_list:
        embeddings = []
        for word in ng.split():
            synsets = wn.synsets(word)
            if len(synsets) > 1:
                glosses = [s.definition() for s in synsets]
                e = model.encode(glosses, convert_to_tensor=True)
                embeddings.append(e.mean(dim=0))  # average embeddings for this word
        if embeddings:
            sim_matrix = util.cos_sim(torch.stack(embeddings), torch.stack(embeddings))
            score = 1 - sim_matrix.mean().item()
            scores.append(score)

    return sum(scores)/len(scores) if scores else 0

# ===============================
# Apply to your DataFrame
# ===============================
# Example: df1 has 'Combined_News' column
df1['Ambiguous_Bigrams'] = df1['Combined_News'].apply(lambda x: check_ngram_ambiguity(x, n=2))
df1['Bigram_Ambiguity_Score'] = df1['Combined_News'].apply(lambda x: ngram_semantic_score(x, n=2))

# ===============================
# Preview
# ===============================
print(df1[['Combined_News', 'Ambiguous_Bigrams', 'Bigram_Ambiguity_Score']].head())

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

                                       Combined_News  \
0  egyptian salafi preacher say rape sexually har...   
1  pope pope benedict xvi resign pope resign end ...   
2  iran urge destroy atomic weapon nkorea nuclear...   
3  european union begin formal talk freetrade agr...   
4  totally drugresistant tb uncovered outbreak un...   

                                   Ambiguous_Bigrams  Bigram_Ambiguity_Score  
0  [rape harass, harass woman, woman protester, t...                0.315011  
1  [resign resign, resign end, end month, unexpec...                0.290107  
2  [urge destroy, destroy atomic, atomic weapon, ...                0.302256  
3  [begin formal, formal talk, talk freetrade, fr...                0.302476  
4  [drugresistant uncovered, uncovered outbreak, ...                0.249624  


In [13]:
companies = df2["Name"].unique()
print(companies)

['AAL' 'AAPL' 'AAP' 'ABBV' 'ABC' 'ABT' 'ACN' 'ADBE' 'ADI' 'ADM' 'ADP'
 'ADSK' 'ADS' 'AEE' 'AEP' 'AES' 'AET' 'AFL' 'AGN' 'AIG' 'AIV' 'AIZ' 'AJG'
 'AKAM' 'ALB' 'ALGN' 'ALK' 'ALL' 'ALXN' 'AMAT' 'AMD' 'AME' 'AMGN' 'AMG'
 'AMP' 'AMT' 'AMZN' 'ANDV' 'ANSS' 'ANTM' 'AON' 'AOS' 'APA' 'APC' 'APD'
 'APH' 'ARE' 'ARNC' 'ATVI' 'AVB' 'AVGO' 'AVY' 'AWK' 'AXP' 'AYI' 'AZO' 'A'
 'BAC' 'BAX' 'BA' 'BBT' 'BBY' 'BDX' 'BEN' 'BF.B' 'BIIB' 'BK' 'BLK' 'BLL'
 'BMY' 'BRK.B' 'BSX' 'BWA' 'BXP' 'CAG' 'CAH' 'CAT' 'CA' 'CBG' 'CBOE' 'CBS'
 'CB' 'CCI' 'CCL' 'CDNS' 'CELG' 'CERN' 'CF' 'CHD' 'CHK' 'CHRW' 'CHTR'
 'CINF' 'CI' 'CLX' 'CL' 'CMA' 'CMCSA' 'CME' 'CMG' 'CMI' 'CMS' 'CNC' 'CNP'
 'COF' 'COG' 'COL' 'COO' 'COP' 'COST' 'CPB' 'CRM' 'CSCO' 'CSX' 'CTAS'
 'CTL' 'CTSH' 'CTXS' 'CVS' 'CVX' 'CXO' 'C' 'DAL' 'DE' 'DFS' 'DGX' 'DG'
 'DHI' 'DISCA' 'DISCK' 'DISH' 'DIS' 'DLR' 'DLTR' 'DOV' 'DPS' 'DRE' 'DRI'
 'DTE' 'DUK' 'DVA' 'DVN' 'D' 'EA' 'EBAY' 'ECL' 'ED' 'EFX' 'EIX' 'EL' 'EMN'
 'EMR' 'EOG' 'EQIX' 'EQR' 'EQT' 'ESRX' 'ESS' 'ES' 'ETFC' '

In [14]:
df1['Date'] = pd.to_datetime(df1['Date'], dayfirst=True, errors='coerce')
df2['Date'] = pd.to_datetime(df2['Date'], dayfirst=True, errors='coerce')

In [15]:
for company in companies:
    temp = df2[df2["Name"] == company][["Date", "price_label"]]
    df1 = df1.merge(temp.rename(columns={"price_label": f"{company}"}),
                    on="Date", how="left")


In [16]:
df1

,Date,Combined_News,Ambiguous_Bigrams,Bigram_Ambiguity_Score,AAL,AAPL,AAP,ABBV,ABC,ABT,ACN,ADBE,ADI,ADM,ADP,ADSK,ADS,AEE,AEP,AES,AET,AFL,AGN,AIG,AIV,AIZ,AJG,AKAM,ALB,ALGN,ALK,ALL,ALXN,AMAT,AMD,AME,AMGN,AMG,AMP,AMT,AMZN,ANDV,ANSS,ANTM,AON,AOS,APA,APC,APD,APH,ARE,ARNC,ATVI,AVB,AVGO,AVY,AWK,AXP,AYI,AZO,A,BAC,BAX,BA,BBT,BBY,BDX,BEN,BF.B,BIIB,BK,BLK,BLL,BMY,BRK.B,BSX,BWA,BXP,CAG,CAH,CAT,CA,CBG,CBOE,CBS,CB,CCI,CCL,CDNS,CELG,CERN,CF,CHD,CHK,CHRW,CHTR,CINF,CI,CLX,CL,CMA,CMCSA,CME,CMG,CMI,CMS,CNC,CNP,COF,COG,COL,COO,COP,COST,CPB,CRM,CSCO,CSX,CTAS,CTL,CTSH,CTXS,CVS,CVX,CXO,C,DAL,DE,DFS,DGX,DG,DHI,DISCA,DISCK,DISH,DIS,DLR,DLTR,DOV,DPS,DRE,DRI,DTE,DUK,DVA,DVN,D,EA,EBAY,ECL,ED,EFX,EIX,EL,EMN,EMR,EOG,EQIX,EQR,EQT,ESRX,ESS,ES,ETFC,ETN,ETR,EW,EXC,EXPD,EXPE,EXR,FAST,FBHS,FB,FCX,FDX,FE,FFIV,FISV,FIS,FITB,FLIR,FLR,FLS,FL,FMC,FRT,FTI,F,GD,GE,GGP,GILD,GIS,GLW,GM,GOOGL,GPC,GPN,GPS,GRMN,GS,GT,GWW,HAL,HAS,HBAN,HBI,HCA,HCN,HCP,HD,HES,HIG,HII,HOG,HOLX,HON,HP,HRB,HRL,HRS,HSIC,HST,HSY,HUM,IBM,ICE,IDXX,IFF,ILMN,INCY,INTC,INTU,IPG,IP,IRM,IR,ISRG,ITW,IT,IVZ,JBHT,JCI,JEC,JNJ,JNPR,JPM,JWN,KEY,KIM,KLAC,KMB,KMI,KMX,KORS,KO,KR,KSS,KSU,K,LB,LEG,LEN,LH,LKQ,LLL,LLY,LMT,LNC,LNT,LOW,LRCX,LUK,LUV,LYB,L,MAA,MAC,MAR,MAS,MAT,MA,MCD,MCHP,MCK,MCO,MDLZ,MDT,MET,MGM,MHK,MKC,MLM,MMC,MMM,MNST,MON,MOS,MO,MPC,MRK,MRO,MSFT,MSI,MS,MTB,MTD,MU,MYL,M,NBL,NCLH,NDAQ,NEE,NEM,NFLX,NFX,NI,NKE,NLSN,NOC,NOV,NRG,NSC,NTAP,NTRS,NUE,NVDA,NWL,OKE,OMC,ORCL,ORLY,OXY,O,PAYX,PBCT,PCAR,PCG,PCLN,PDCO,PEG,PEP,PFE,PFG,PGR,PG,PHM,PH,PKG,PKI,PLD,PM,PNC,PNR,PNW,PPG,PPL,PRGO,PRU,PSA,PSX,PVH,PWR,PXD,PX,QCOM,RCL,REGN,REG,RE,RF,RHI,RHT,RJF,RL,RMD,ROK,ROP,ROST,RRC,RSG,RTN,SBAC,SBUX,SCG,SCHW,SEE,SHW,SIG,SJM,SLB,SLG,SNA,SNI,SNPS,SO,SPGI,SPG,SRCL,SRE,STI,STT,STX,STZ,SWKS,SWK,SYK,SYMC,SYY,TAP,TDG,TEL,TGT,TIF,TJX,TMK,TMO,TPR,TRIP,TROW,TRV,TSCO,TSN,TSS,TWX,TXN,TXT,T,UAA,UAL,UDR,UHS,ULTA,UNH,UNM,UNP,UPS,URI,USB,UTX,VAR,VFC,VIAB,VLO,VMC,VNO,VRSK,VRSN,VRTX,VTR,VZ,V,WAT,WBA,WDC,WEC,WFC,WHR,WMB,WMT,WM,WU,WYNN,WYN,WY,XEC,XEL,XLNX,XL,XOM,XRAY,XRX,XYL,YUM,ZBH,ZION,ZTS
0,2013-02-08,egyptian salafi preacher say rape sexually har...,"[rape harass, harass woman, woman protester, t...",0.315011,0,1,1,0,1,1,1,1,1,0,1,1,1,0,1,0,1,0,1,1,1,1,0,0,1,1,1,1,1,1,1,1,1,0,1,1,1,1,0,1,1,1,1,1,1,1,0,1,1,1,1,1,1,0,1,1,1,0,1,0,0,0,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,0,0,1,1,1,1,1,0,1,1,0,1,1,1,1,1,1,0,0,1,0,1,0,1,1,1,1,0,0,1,0,0,1,0,1,1,1,1,1,1,1,0,1,0,1,1,0,1,1,1,1,1,1,0,1,0,1,1,1,1,1,0,1,1,1,1,0,0,1,0,1,1,1,1,1,0,0,0,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,0,0,1,0,1,1,1,1,1,0,1,1,1,1,0,1,1,1,1,0,1,1,0,1,1,1,0,1,0,0,1,0,1,1,0,1,1,0,1,1,1,0,0,1,1,1,1,0,1,1,1,1,0,0,0,1,0,0,1,1,0,0,0,0,0,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,0,1,1,1,0,1,0,0,1,0,1,1,0,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,0,0,0,1,1,1,0,0,0,0,0,1,0,1,1,0,1,1,1,1,1,1,1,1,1,1,1,0,0,1,1,1,1,0,0,0,0,1,1,0,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,0,0,1,1,1,1,1,1,1,0,0,0,0,1,0,1,1,1,1,1,1,1,1,1,0,1,1,0,1,1,1,1,1,1,0,0,1,0,1,1,1,1,1,1,0,1,1,1,1,0,1,1,0,1,1,1,1,1,1,0,1,1,1,1,1,1,0,0,1,1,1,1,1,1,1,1,1,1,0,0,0,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1
1,2013-02-11,pope pope benedict xvi resign pope resign end ...,"[resign resign, resign end, end month, unexpec...",0.290107,0,1,0,0,0,0,0,0,1,1,0,0,0,1,1,1,0,0,0,1,1,0,1,0,1,1,0,1,0,1,1,1,0,0,1,0,0,1,1,0,1,0,0,0,0,0,0,0,1,1,1,1,0,1,1,1,0,1,0,0,0,1,1,1,0,0,1,1,0,1,0,0,1,1,0,1,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,1,1,0,0,1,1,1,0,1,0,0,0,0,0,0,1,0,1,1,1,1,0,0,0,0,1,1,1,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1,0,0,1,1,0,0,1,0,1,0,1,1,1,1,1,0,0,1,1,1,1,1,1,1,0,0,1,1,0,0,0,1,1,0,1,1,1,1,0,1,1,0,1,0,1,0,0,1,0,1,1,1,1,0,1,1,0,1,1,1,0,0,0,0,0,1,1,0,0,1,1,0,1,0,0,1,0,1,1,1,1,0,0,1,0,0,0,1,1,0,0,0,0,0,0,1,0,1,0,1,0,1,0,1,0,0,0,0,1,1,0,0,0,0,1,1,1,0,0,0,0,0,1,1,0,1,1,0,1,0,0,0,0,1,0,1,1,1,0,1,1,1,1,1,1,1,0,1,1,0,0,1,1,1,0,1,0,1,0,1,1,1,1,0,1,1,1,0,0,1,1,0,0,0,1,0,0,1,0,1,1,0,1,1,1,0,1,1,0,1,0,1,1,1,1,0,0,0,1,0,1,1,1,1,0,1,0,1,0,0,1,1,1,1,1,1,0,0,1,0,0,0,0,1,1,0,1,1,0,0,0,0,0,0,1,0,1,0,1,1,0,0,1,1,0,1,0,0,0,0,0,1,0,1,1,1,0,0,1,

In [17]:
import torch
import numpy as np
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from tqdm import tqdm

# load tokenizer + model
model_name = "cardiffnlp/twitter-roberta-base-sentiment-latest"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

# device handling
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

# batched scoring function
def roberta_batch_scores(texts, batch_size=32, max_length=512):
    """
    texts: list of strings
    returns: numpy array shape (len(texts), 3) with [neg, neu, pos] probs
    """
    all_probs = []
    for i in tqdm(range(0, len(texts), batch_size), desc="RoBERTa batches"):
        batch_texts = texts[i:i+batch_size]
        enc = tokenizer(batch_texts, return_tensors="pt", padding=True,
                        truncation=True, max_length=max_length)
        enc = {k: v.to(device) for k, v in enc.items()}
        with torch.no_grad():
            out = model(**enc)
            probs = torch.softmax(out.logits, dim=1).cpu().numpy()  # shape (batch_size, 3)
        all_probs.append(probs)
    return np.vstack(all_probs)

# Define bins for your scores
def bin_ambiguity(score):
    if score < 0.3:
        return "[LOW_AMBIGUITY]"
    elif score < 0.6:
        return "[MEDIUM_AMBIGUITY]"
    else:
        return "[HIGH_AMBIGUITY]"

# Create a new column with text + ambiguity token
df1['text_with_ambiguity'] = df1.apply(
    lambda row: f"{row['Combined_News']} {bin_ambiguity(row['Bigram_Ambiguity_Score'])}", axis=1
)

# Run RoBERTa on the new column
texts = df1['text_with_ambiguity'].astype(str).tolist()
probs = roberta_batch_scores(texts, batch_size=32, max_length=512)

# Attach back to DataFrame
probs_df = pd.DataFrame(probs, columns=['roberta_neg','roberta_neu','roberta_pos'], index=df1.index)
merge = pd.concat([df1, probs_df], axis=1)
merge['roberta_score'] = probs_df.max(axis=1)
merge['roberta_label'] = probs_df.idxmax(axis=1)

# ✅ Check sample
display(merge[['Combined_News', 'Bigram_Ambiguity_Score', 'text_with_ambiguity',
               'roberta_neg','roberta_neu','roberta_pos','roberta_score','roberta_label']].head())



config.json:   0%|          | 0.00/929 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/501M [00:00<?, ?B/s]

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
RoBERTa batches:   0%|          | 0/27 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/501M [00:00<?, ?B/s]

RoBERTa batches: 100%|██████████| 27/27 [00:24<00:00,  1.12it/s]


,Combined_News,Bigram_Ambiguity_Score,text_with_ambiguity,roberta_neg,roberta_neu,roberta_pos,roberta_score,roberta_label
0,egyptian salafi preacher say rape sexually har...,0.315011,egyptian salafi preacher say rape sexually har...,0.755523,0.232934,0.011543,0.755523,roberta_neg
1,pope pope benedict xvi resign pope resign end ...,0.290107,pope pope benedict xvi resign pope resign end ...,0.614470,0.367851,0.017679,0.614470,roberta_neg
2,iran urge destroy atomic weapon nkorea nuclear...,0.302256,iran urge destroy atomic weapon nkorea nuclear...,0.710625,0.273520,0.015855,0.710625,roberta_neg
3,european union begin formal talk freetrade agr...,0.302476,european union begin formal talk freetrade agr...,0.436779,0.536104,0.027117,0.536104,roberta_neu
4,totally drugresistant tb uncovered outbreak un...,0.249624,totally drugresistant tb uncovered outbreak un...,0.756200,0.231858,0.011942,0.756200,roberta_neg


In [18]:
def getSIA(text):
    sia = SentimentIntensityAnalyzer()
    sentiment = sia.polarity_scores(text)
    return sentiment

In [19]:
neg=[]
pos=[]
neu=[]
SIA=0

for i in range (0, len(merge['Combined_News'])):
    SIA= getSIA(merge['Combined_News'][i])
    neg.append(SIA['neg'])
    pos.append(SIA['pos'])
    neu.append(SIA['neu'])

merge['neg']= neg
merge['pos']=pos
merge['neu']=neu

In [20]:
import numpy as np

def combine_roberta_vader(row, weight_roberta=0.7, weight_vader=0.3):
    neg = weight_roberta * row['roberta_neg'] + weight_vader * row['neg']
    neu = weight_roberta * row['roberta_neu'] + weight_vader * row['neu']
    pos = weight_roberta * row['roberta_pos'] + weight_vader * row['pos']

    scores = [neg, neu, pos]
    labels = ['negative', 'neutral', 'positive']
    final_label = labels[np.argmax(scores)]

    return final_label, neg, neu, pos

merge[['final_label', 'final_neg', 'final_neu', 'final_pos']] = merge.apply(
    lambda x: combine_roberta_vader(x), axis=1, result_type="expand"
)


In [21]:
merge=merge.drop(columns=["roberta_neg","roberta_neu","roberta_pos","roberta_score","roberta_label","neg","pos","neu"],axis=1)
merge

,Date,Combined_News,Ambiguous_Bigrams,Bigram_Ambiguity_Score,AAL,AAPL,AAP,ABBV,ABC,ABT,ACN,ADBE,ADI,ADM,ADP,ADSK,ADS,AEE,AEP,AES,AET,AFL,AGN,AIG,AIV,AIZ,AJG,AKAM,ALB,ALGN,ALK,ALL,ALXN,AMAT,AMD,AME,AMGN,AMG,AMP,AMT,AMZN,ANDV,ANSS,ANTM,AON,AOS,APA,APC,APD,APH,ARE,ARNC,ATVI,AVB,AVGO,AVY,AWK,AXP,AYI,AZO,A,BAC,BAX,BA,BBT,BBY,BDX,BEN,BF.B,BIIB,BK,BLK,BLL,BMY,BRK.B,BSX,BWA,BXP,CAG,CAH,CAT,CA,CBG,CBOE,CBS,CB,CCI,CCL,CDNS,CELG,CERN,CF,CHD,CHK,CHRW,CHTR,CINF,CI,CLX,CL,CMA,CMCSA,CME,CMG,CMI,CMS,CNC,CNP,COF,COG,COL,COO,COP,COST,CPB,CRM,CSCO,CSX,CTAS,CTL,CTSH,CTXS,CVS,CVX,CXO,C,DAL,DE,DFS,DGX,DG,DHI,DISCA,DISCK,DISH,DIS,DLR,DLTR,DOV,DPS,DRE,DRI,DTE,DUK,DVA,DVN,D,EA,EBAY,ECL,ED,EFX,EIX,EL,EMN,EMR,EOG,EQIX,EQR,EQT,ESRX,ESS,ES,ETFC,ETN,ETR,EW,EXC,EXPD,EXPE,EXR,FAST,FBHS,FB,FCX,FDX,FE,FFIV,FISV,FIS,FITB,FLIR,FLR,FLS,FL,FMC,FRT,FTI,F,GD,GE,GGP,GILD,GIS,GLW,GM,GOOGL,GPC,GPN,GPS,GRMN,GS,GT,GWW,HAL,HAS,HBAN,HBI,HCA,HCN,HCP,HD,HES,HIG,HII,HOG,HOLX,HON,HP,HRB,HRL,HRS,HSIC,HST,HSY,HUM,IBM,ICE,IDXX,IFF,ILMN,INCY,INTC,INTU,IPG,IP,IRM,IR,ISRG,ITW,IT,IVZ,JBHT,JCI,JEC,JNJ,JNPR,JPM,JWN,KEY,KIM,KLAC,KMB,KMI,KMX,KORS,KO,KR,KSS,KSU,K,LB,LEG,LEN,LH,LKQ,LLL,LLY,LMT,LNC,LNT,LOW,LRCX,LUK,LUV,LYB,L,MAA,MAC,MAR,MAS,MAT,MA,MCD,MCHP,MCK,MCO,MDLZ,MDT,MET,MGM,MHK,MKC,MLM,MMC,MMM,MNST,MON,MOS,MO,MPC,MRK,MRO,MSFT,MSI,MS,MTB,MTD,MU,MYL,M,NBL,NCLH,NDAQ,NEE,NEM,NFLX,NFX,NI,NKE,NLSN,NOC,NOV,NRG,NSC,NTAP,NTRS,NUE,NVDA,NWL,OKE,OMC,ORCL,ORLY,OXY,O,PAYX,PBCT,PCAR,PCG,PCLN,PDCO,PEG,PEP,PFE,PFG,PGR,PG,PHM,PH,PKG,PKI,PLD,PM,PNC,PNR,PNW,PPG,PPL,PRGO,PRU,PSA,PSX,PVH,PWR,PXD,PX,QCOM,RCL,REGN,REG,RE,RF,RHI,RHT,RJF,RL,RMD,ROK,ROP,ROST,RRC,RSG,RTN,SBAC,SBUX,SCG,SCHW,SEE,SHW,SIG,SJM,SLB,SLG,SNA,SNI,SNPS,SO,SPGI,SPG,SRCL,SRE,STI,STT,STX,STZ,SWKS,SWK,SYK,SYMC,SYY,TAP,TDG,TEL,TGT,TIF,TJX,TMK,TMO,TPR,TRIP,TROW,TRV,TSCO,TSN,TSS,TWX,TXN,TXT,T,UAA,UAL,UDR,UHS,ULTA,UNH,UNM,UNP,UPS,URI,USB,UTX,VAR,VFC,VIAB,VLO,VMC,VNO,VRSK,VRSN,VRTX,VTR,VZ,V,WAT,WBA,WDC,WEC,WFC,WHR,WMB,WMT,WM,WU,WYNN,WYN,WY,XEC,XEL,XLNX,XL,XOM,XRAY,XRX,XYL,YUM,ZBH,ZION,ZTS,text_with_ambiguity,final_label,final_neg,final_neu,final_pos
0,2013-02-08,egyptian salafi preacher say rape sexually har...,"[rape harass, harass woman, woman protester, t...",0.315011,0,1,1,0,1,1,1,1,1,0,1,1,1,0,1,0,1,0,1,1,1,1,0,0,1,1,1,1,1,1,1,1,1,0,1,1,1,1,0,1,1,1,1,1,1,1,0,1,1,1,1,1,1,0,1,1,1,0,1,0,0,0,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,0,0,1,1,1,1,1,0,1,1,0,1,1,1,1,1,1,0,0,1,0,1,0,1,1,1,1,0,0,1,0,0,1,0,1,1,1,1,1,1,1,0,1,0,1,1,0,1,1,1,1,1,1,0,1,0,1,1,1,1,1,0,1,1,1,1,0,0,1,0,1,1,1,1,1,0,0,0,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,0,0,1,0,1,1,1,1,1,0,1,1,1,1,0,1,1,1,1,0,1,1,0,1,1,1,0,1,0,0,1,0,1,1,0,1,1,0,1,1,1,0,0,1,1,1,1,0,1,1,1,1,0,0,0,1,0,0,1,1,0,0,0,0,0,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,0,1,1,1,0,1,0,0,1,0,1,1,0,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,0,0,0,1,1,1,0,0,0,0,0,1,0,1,1,0,1,1,1,1,1,1,1,1,1,1,1,0,0,1,1,1,1,0,0,0,0,1,1,0,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,0,0,1,1,1,1,1,1,1,0,0,0,0,1,0,1,1,1,1,1,1,1,1,1,0,1,1,0,1,1,1,1,1,1,0,0,1,0,1,1,1,1,1,1,0,1,1,1,1,0,1,1,0,1,1,1,1,1,1,0,1,1,1,1,1,1,0,0,1,1,1,1,1,1,1,1,1,1,0,0,0,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,egyptian salafi preacher say rape sexually har...,negative,0.643766,0.335254,0.020980
1,2013-02-11,pope pope benedict xvi resign pope resign end ...,"[resign resign, resign end, end month, unexpec...",0.290107,0,1,0,0,0,0,0,0,1,1,0,0,0,1,1,1,0,0,0,1,1,0,1,0,1,1,0,1,0,1,1,1,0,0,1,0,0,1,1,0,1,0,0,0,0,0,0,0,1,1,1,1,0,1,1,1,0,1,0,0,0,1,1,1,0,0,1,1,0,1,0,0,1,1,0,1,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,1,1,0,0,1,1,1,0,1,0,0,0,0,0,0,1,0,1,1,1,1,0,0,0,0,1,1,1,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1,0,0,1,1,0,0,1,0,1,0,1,1,1,1,1,0,0,1,1,1,1,1,1,1,0,0,1,1,0,0,0,1,1,0,1,1,1,1,0,1,1,0,1,0,1,0,0,1,0,1,1,1,1,0,1,1,0,1,1,1,0,0,0,0,0,1,1,0,0,1,1,0,1,0,0,1,0,1,1,1,1,0,0,1,0,0,0,1,1,0,0,0,0,0,0,1,0,1,0,1,0,1,0,1,0,0,0,0,1,1,0,0,0,0,1,1,1,0,0,0,0,0,1,1,0,1,1,0,1,0,0,0,0,1,0,1,1,1,0,1,1,1,1,1,1,1,0,1,1,0,0,1,1,1,0,1,0,1,0,1,1,1,1,0,1,1,1,0,0,1,1,0,0,0,1,0,0,1,0,1,1,0,1,1,1,

In [22]:
import pandas as pd
company_sectors=pd.read_csv("/content/sp500_companies.csv")
company_sectors

,Symbol,Name,Sector
0,AAPL,Apple Inc.,Technology
1,NVDA,NVIDIA Corporation,Technology
2,MSFT,Microsoft Corporation,Technology
3,AMZN,"Amazon.com, Inc.",Consumer Cyclical
4,GOOGL,Alphabet Inc.,Communication Services
...,...,...,...
470,XLNX,"Xilinx, Inc.",Technology
471,XL,XL Group Ltd.,Financial Services
472,XRAY,DENTSPLY SIRONA Inc.,Healthcare
473,XRX,Xerox Corporation,Technology


In [23]:
sector_dict = company_sectors.groupby("Sector")["Symbol"].apply(list).to_dict()
sector_dict

{'Basic Materials': ['SHW',
  'ECL',
  'APD',
  'FCX',
  'NEM',
  'VMC',
  'MLM',
  'PPG',
  'NUE',
  'LYB',
  'IFF',
  'CF',
  'EMN',
  'ALB',
  'MOS',
  'FMC',
  'MON',
  'PX'],
 'Communication Services': ['GOOGL',
  'NFLX',
  'DIS',
  'VZ',
  'T',
  'CMCSA',
  'CHTR',
  'EA',
  'OMC',
  'IPG',
  'ATVI',
  'CBS',
  'CTL',
  'DISCA',
  'DISCK',
  'DISH',
  'FB',
  'SNI',
  'TRIP',
  'TWX',
  'VIAB'],
 'Consumer Cyclical': ['AMZN',
  'HD',
  'MCD',
  'LOW',
  'TJX',
  'NKE',
  'SBUX',
  'CMG',
  'MAR',
  'ORLY',
  'RCL',
  'GM',
  'AZO',
  'ROST',
  'DHI',
  'F',
  'LEN',
  'YUM',
  'CCL',
  'EBAY',
  'TSCO',
  'EXPE',
  'PHM',
  'DRI',
  'PKG',
  'ULTA',
  'IP',
  'BBY',
  'GPC',
  'AVY',
  'TPR',
  'RL',
  'KMX',
  'NCLH',
  'MGM',
  'WYNN',
  'LKQ',
  'HAS',
  'MHK',
  'BWA',
  'AAP',
  'BLL',
  'FL',
  'GPS',
  'GT',
  'HBI',
  'HOG',
  'HRB',
  'JWN',
  'KORS',
  'KSS',
  'LB',
  'LEG',
  'MAT',
  'M',
  'NWL',
  'PCLN',
  'PVH',
  'SEE',
  'SIG',
  'TIF',
  'UAA',
  'VFC',
  'WHR

In [ ]:
import os
import openai
import pandas as pd
import numpy as np
from tqdm import tqdm
from openai import OpenAI
import getpass

# OpenAI setup

YOUR_API_KEY_HERE = getpass.getpass("Enter your OpenAI API key: ")
client = OpenAI(api_key=YOUR_API_KEY_HERE)

# Build date x sector dataset (rule-based from company votes)

def build_sector_date_df(merge_df, sector_dict):
    dates = sorted(merge_df['Date'].dropna().unique())
    rows = []
    for dt in tqdm(dates, desc="building sector-date rows"):
        row_news = merge_df.loc[merge_df['Date'] == dt, 'Combined_News']
        combined_news_text = " ".join(row_news.astype(str).tolist())

        for sector, symbols in sector_dict.items():
            company_cols = [s for s in symbols if s in merge_df.columns]
            if not company_cols:
                continue

            # Get values for this date
            subset = merge_df.loc[merge_df["Date"] == dt, company_cols]
            if subset.empty:
                continue

            # Count votes
            ones = (subset == 1).sum().sum()
            zeros = (subset == 0).sum().sum()

            if ones > zeros:
                movement = "up"; movement_label = 1
            elif zeros > ones:
                movement = "down"; movement_label = 0
            else:
                movement = "neutral"; movement_label = 2

            rows.append({
                "Date": dt,
                "Sector": sector,
                "Combined_News": combined_news_text,
                "ones": int(ones),
                "zeros": int(zeros),
                "movement": movement,
                "movement_label": movement_label,
                "n_companies": len(company_cols)
            })
    sector_date_df = pd.DataFrame(rows)
    return sector_date_df


sector_date_df = build_sector_date_df(merge, sector_dict)
print("Sector-date table:", sector_date_df.shape)
print(sector_date_df.head())

# explain a given date & sector (rule-based)

def explain_date_sector(date, sector, openai_model="gpt-4o-mini", openai_max_tokens=400):
    df_row = sector_date_df[(sector_date_df['Date'] == pd.to_datetime(date)) & (sector_date_df['Sector'] == sector)]
    if df_row.shape[0] == 0:
        return {"error": "No data for this date+sector"}

    df_row = df_row.iloc[0]
    text = df_row['Combined_News']

    movement = df_row['movement']
    n_companies = df_row['n_companies']
    ones = df_row['ones']
    zeros = df_row['zeros']

    prompt = f"""
You are a careful market journalist and data scientist.
For the given Date: {pd.to_datetime(date).strftime('%Y-%m-%d')} and Sector: {sector}:
- Rule-based voting shows {ones} companies voted UP and {zeros} voted DOWN (total {n_companies} companies).
- The final sector movement = {movement}.

Here are the combined news headlines for that date:
\"\"\"{text}\"\"\"

Write a concise (3–6 sentence) explanation for WHY the sector likely moved {movement} on that date.
Also provide 3 short bullet points listing the strongest pieces of textual evidence.
Be explicit about uncertainty if n_companies < 3.
"""

    try:
        resp = client.chat.completions.create(
            model=openai_model,
            messages=[
                {"role": "system", "content": "You are a helpful assistant that explains movements in financial sectors using news headlines and company votes."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=openai_max_tokens,
            temperature=0.0
        )
        explanation = resp.choices[0].message.content.strip()
    except Exception as e:
        explanation = f"OpenAI call failed: {e}"

    return {
        "date": pd.to_datetime(date),
        "sector": sector,
        "movement": movement,
        "ones": int(ones),
        "zeros": int(zeros),
        "n_companies": int(n_companies),
        "explanation": explanation,
        "combined_news": text
    }

# Generate explanations

def explain_all_sectors_on_date(date, sectors=None, save_csv=None):
    results = []
    sectors_to_explain = sectors if sectors is not None else sector_date_df['Sector'].unique()
    for sector in tqdm(sectors_to_explain, desc=f"Explaining sectors for {date}"):
        res = explain_date_sector(date, sector)
        if "error" not in res:
            results.append(res)
    df_out = pd.DataFrame(results)
    if save_csv:
        df_out.to_csv(save_csv, index=False)
    return df_out

def explain_all_dates_all_sectors(limit_dates=None, save_csv=None):
    all_dates = sorted(sector_date_df['Date'].unique())
    if limit_dates:
        all_dates = all_dates[:limit_dates]
    all_results = []
    for dt in tqdm(all_dates, desc="All dates"):
        df_dt = explain_all_sectors_on_date(dt)
        all_results.append(df_dt)
    final = pd.concat(all_results, ignore_index=True)
    if save_csv:
        final.to_csv(save_csv, index=False)
    return final

available_dates = sorted(sector_date_df['Date'].drop_duplicates())

while True:
    user_input = input(f"Enter a date (YYYY-MM-DD). Available range: {available_dates[0].date()} to {available_dates[-1].date()} : ")
    try:
        user_date = pd.to_datetime(user_input)
    except Exception:
        print("Invalid format. Please enter in YYYY-MM-DD format.")
        continue

    if user_date in available_dates:
        sample_date = user_date
        break
    else:
        print("Date not found in dataset. Please try again.")

explanations_df = explain_all_sectors_on_date(sample_date, save_csv="sector_explanations_sample_date.csv")
display(explanations_df[['date','sector','movement','ones','zeros','n_companies','explanation']].head())


Enter your OpenAI API key: ··········


building sector-date rows: 100%|██████████| 856/856 [00:12<00:00, 70.02it/s]


Sector-date table: (9416, 8)
        Date                  Sector  \
0 2013-02-08         Basic Materials   
1 2013-02-08  Communication Services   
2 2013-02-08       Consumer Cyclical   
3 2013-02-08      Consumer Defensive   
4 2013-02-08                  Energy   

                                       Combined_News  ones  zeros movement  \
0  egyptian salafi preacher say rape sexually har...    16      2       up   
1  egyptian salafi preacher say rape sexually har...    17      4       up   
2  egyptian salafi preacher say rape sexually har...    49     16       up   
3  egyptian salafi preacher say rape sexually har...    19     14       up   
4  egyptian salafi preacher say rape sexually har...    27      4       up   

   movement_label  n_companies  
0               1           18  
1               1           21  
2               1           65  
3               1           33  
4               1           31  
Enter a date (YYYY-MM-DD). Available range: 2013-02-08 to 2016-

Explaining sectors for 2015-06-17 00:00:00: 100%|██████████| 11/11 [02:50<00:00, 15.48s/it]


,date,sector,movement,ones,zeros,n_companies,explanation
0,2015-06-17,Basic Materials,neutral,9,9,18,"On June 17, 2015, the Basic Materials sector e..."
1,2015-06-17,Communication Services,up,13,8,21,"On June 17, 2015, the Communication Services s..."
2,2015-06-17,Consumer Cyclical,up,41,24,65,"On June 17, 2015, the Consumer Cyclical sector..."
3,2015-06-17,Consumer Defensive,up,28,5,33,"On June 17, 2015, the Consumer Defensive secto..."
4,2015-06-17,Energy,down,3,28,31,"On June 17, 2015, the Energy sector likely mov..."


In [ ]:
import pandas as pd
import plotly.graph_objects as go

print("Available companies:", companies)

# Function to plot candlestick for one company
def plot_candlestick(company_name, start_date=None, end_date=None):
    company_df = df2[df2['Name'] == company_name].copy()
    if start_date:
        company_df = company_df[company_df['Date'] >= pd.to_datetime(start_date)]
    if end_date:
        company_df = company_df[company_df['Date'] <= pd.to_datetime(end_date)]

    fig = go.Figure(data=[go.Candlestick(
        x=company_df['Date'],
        open=company_df['Open'],
        high=company_df['High'],
        low=company_df['Low'],
        close=company_df['Close'],
        name=company_name
    )])

    fig.update_layout(
        title=f"Candlestick Chart for {company_name}",
        xaxis_title="Date",
        yaxis_title="Price (USD)",
        xaxis_rangeslider_visible=True,
        template="plotly_dark"
    )

    fig.show()

while True:
    comp = input(f"Enter a company name from dataset {list(companies)}...: ")
    if comp in companies:
        break
    else:
        print("Company not found. Please enter a valid company from the dataset.")
plot_candlestick(comp, start_date="2008-08-08", end_date="2016-07-01")

Available companies: ['AAL' 'AAPL' 'AAP' 'ABBV' 'ABC' 'ABT' 'ACN' 'ADBE' 'ADI' 'ADM' 'ADP'
 'ADSK' 'ADS' 'AEE' 'AEP' 'AES' 'AET' 'AFL' 'AGN' 'AIG' 'AIV' 'AIZ' 'AJG'
 'AKAM' 'ALB' 'ALGN' 'ALK' 'ALL' 'ALXN' 'AMAT' 'AMD' 'AME' 'AMGN' 'AMG'
 'AMP' 'AMT' 'AMZN' 'ANDV' 'ANSS' 'ANTM' 'AON' 'AOS' 'APA' 'APC' 'APD'
 'APH' 'ARE' 'ARNC' 'ATVI' 'AVB' 'AVGO' 'AVY' 'AWK' 'AXP' 'AYI' 'AZO' 'A'
 'BAC' 'BAX' 'BA' 'BBT' 'BBY' 'BDX' 'BEN' 'BF.B' 'BIIB' 'BK' 'BLK' 'BLL'
 'BMY' 'BRK.B' 'BSX' 'BWA' 'BXP' 'CAG' 'CAH' 'CAT' 'CA' 'CBG' 'CBOE' 'CBS'
 'CB' 'CCI' 'CCL' 'CDNS' 'CELG' 'CERN' 'CF' 'CHD' 'CHK' 'CHRW' 'CHTR'
 'CINF' 'CI' 'CLX' 'CL' 'CMA' 'CMCSA' 'CME' 'CMG' 'CMI' 'CMS' 'CNC' 'CNP'
 'COF' 'COG' 'COL' 'COO' 'COP' 'COST' 'CPB' 'CRM' 'CSCO' 'CSX' 'CTAS'
 'CTL' 'CTSH' 'CTXS' 'CVS' 'CVX' 'CXO' 'C' 'DAL' 'DE' 'DFS' 'DGX' 'DG'
 'DHI' 'DISCA' 'DISCK' 'DISH' 'DIS' 'DLR' 'DLTR' 'DOV' 'DPS' 'DRE' 'DRI'
 'DTE' 'DUK' 'DVA' 'DVN' 'D' 'EA' 'EBAY' 'ECL' 'ED' 'EFX' 'EIX' 'EL' 'EMN'
 'EMR' 'EOG' 'EQIX' 'EQR' 'EQT' 'ESRX

In [ ]:
merge[['final_neg','final_neu','final_pos']].describe()

,final_neg,final_neu,final_pos
count,856.000000,856.000000,856.000000
mean,0.459461,0.493700,0.046834
std,0.130777,0.119204,0.018136
min,0.069694,0.273650,0.015483
25%,0.381392,0.400084,0.036143
50%,0.487528,0.469427,0.043999
75%,0.559441,0.564698,0.053309
max,0.697587,0.812305,0.219832


In [ ]:
import plotly.express as px

merge['Date'] = pd.to_datetime(merge['Date'])

fig = px.line(
    merge,
    x="Date",
    y="Bigram_Ambiguity_Score",
    title="Semantic Ambiguity Score Over Time",
    labels={"Date": "Date", "Bigram_Ambiguity_Score": "Ambiguity Score"}
)

# Add interactive range slider
fig.update_layout(
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(count=7, label="1w", step="day", stepmode="backward"),
                dict(count=1, label="1m", step="month", stepmode="backward"),
                dict(count=6, label="6m", step="month", stepmode="backward"),
                dict(count=1, label="1y", step="year", stepmode="backward"),
                dict(step="all")
            ])
        ),
        rangeslider=dict(visible=True),
        type="date"
    )
)

fig.show()


In [ ]:
import plotly.express as px

# Assuming df is your dataframe with Date, final_pos, final_neg, final_neu
fig = px.line(
    merge,
    x="Date",
    y=max(["final_pos", "final_neg", "final_neu"]),
    title="Sentiment Scores Over Time",
    labels={"value": "Sentiment Score", "variable": "Sentiment Type"}
)

fig.update_traces(mode="lines")
fig.show()


In [ ]:
import os
import openai
import pandas as pd
import numpy as np
from tqdm import tqdm
from openai import OpenAI
import getpass

# OpenAI setup (API key hidden as password input)
YOUR_API_KEY_HERE = getpass.getpass("Enter your OpenAI API key: ")
client = OpenAI(api_key=YOUR_API_KEY_HERE)

# Build date-company dataset (AAPL only)
def build_company_date_df(merge_df, company="AAPL"):
    dates = sorted(merge_df['Date'].dropna().unique())
    rows = []
    for dt in tqdm(dates, desc=f"building rows for {company}"):
        row_news = merge_df.loc[merge_df['Date'] == dt, 'Combined_News']
        combined_news_text = " ".join(row_news.astype(str).tolist())

        if company not in merge_df.columns:
            continue

        subset = merge_df.loc[merge_df["Date"] == dt, [company]]
        if subset.empty:
            continue

        # Movement directly from company column (0/1)
        up_votes = (subset[company] == 1).sum()
        down_votes = (subset[company] == 0).sum()

        if up_votes > down_votes:
            movement = "up"; movement_label = 1
        elif down_votes > up_votes:
            movement = "down"; movement_label = 0
        else:
            movement = "neutral"; movement_label = 2

        rows.append({
            "Date": dt,
            "Company": company,
            "Combined_News": combined_news_text,
            "ones": int(up_votes),
            "zeros": int(down_votes),
            "movement": movement,
            "movement_label": movement_label
        })
    return pd.DataFrame(rows)

aapl_date_df = build_company_date_df(merge, company="AAPL")
print("AAPL-date table:", aapl_date_df.shape)
print(aapl_date_df.head())

# Explain a given date for AAPL
def explain_company_date(date, company="AAPL", openai_model="gpt-4o-mini", openai_max_tokens=400):
    df_row = aapl_date_df[(aapl_date_df['Date'] == pd.to_datetime(date)) & (aapl_date_df['Company'] == company)]
    if df_row.shape[0] == 0:
        return {"error": f"No data for {company} on this date"}

    df_row = df_row.iloc[0]
    text = df_row['Combined_News']

    movement = df_row['movement']
    ones = df_row['ones']
    zeros = df_row['zeros']

    prompt = f"""
You are a careful market journalist and data scientist.
For the given Date: {pd.to_datetime(date).strftime('%Y-%m-%d')} and Company: {company}:
- Rule-based voting shows {ones} UP and {zeros} DOWN.
- The final movement for {company} = {movement}.

Here are the combined news headlines for that date:
\"\"\"{text}\"\"\"

Write a concise (3–6 sentence) explanation for WHY {company} stock likely moved {movement} on that date.
Also provide 3 short bullet points listing the strongest pieces of textual evidence.
"""

    try:
        resp = client.chat.completions.create(
            model=openai_model,
            messages=[
                {"role": "system", "content": "You are a helpful assistant that explains movements in financial markets using news headlines."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=openai_max_tokens,
            temperature=0.0
        )
        explanation = resp.choices[0].message.content.strip()
    except Exception as e:
        explanation = f"OpenAI call failed: {e}"

    return {
        "date": pd.to_datetime(date),
        "company": company,
        "movement": movement,
        "ones": int(ones),
        "zeros": int(zeros),
        "explanation": explanation,
        "combined_news": text
    }

# Generate explanations
def explain_company_on_dates(dates, company="AAPL", save_csv=None):
    results = []
    for dt in tqdm(dates, desc=f"Explaining {company}"):
        res = explain_company_date(dt, company=company)
        if "error" not in res:
            results.append(res)
    df_out = pd.DataFrame(results)
    if save_csv:
        df_out.to_csv(save_csv, index=False)
    return df_out


available_dates = sorted(aapl_date_df['Date'].drop_duplicates())

while True:
    user_input = input(f"Enter a date (YYYY-MM-DD). Available range: {available_dates[0].date()} to {available_dates[-1].date()} : ")
    try:
        user_date = pd.to_datetime(user_input)
    except Exception:
        print("Invalid format. Please enter in YYYY-MM-DD format.")
        continue

    if user_date in available_dates:
        sample_date = user_date
        break
    else:
        print("Date not found in dataset. Please try again.")

explanations_df = explain_company_on_dates([sample_date], company="AAPL", save_csv="aapl_explanations_sample_date.csv")

# Print final results
print(explanations_df[['date','company','movement','ones','zeros','explanation']].head())


Enter your OpenAI API key: ··········


building rows for AAPL: 100%|██████████| 856/856 [00:00<00:00, 858.71it/s]


AAPL-date table: (856, 7)
        Date Company                                      Combined_News  ones  \
0 2013-02-08    AAPL  egyptian salafi preacher say rape sexually har...     1   
1 2013-02-11    AAPL  pope pope benedict xvi resign pope resign end ...     1   
2 2013-02-12    AAPL  iran urge destroy atomic weapon nkorea nuclear...     0   
3 2013-02-13    AAPL  european union begin formal talk freetrade agr...     0   
4 2013-02-14    AAPL  totally drugresistant tb uncovered outbreak un...     1   

   zeros movement  movement_label  
0      0       up               1  
1      0       up               1  
2      1     down               0  
3      1     down               0  
4      0       up               1  
Enter a date (YYYY-MM-DD). Available range: 2013-02-08 to 2016-07-01 : 2015-07-01


Explaining AAPL: 100%|██████████| 1/1 [00:05<00:00,  5.64s/it]

        date company movement  ones  zeros  \
0 2015-07-01    AAPL     down     0      1   

                                         explanation  
0  On July 1, 2015, AAPL's stock likely moved dow...  
